## Load and prep the data for processing


In [64]:
import datetime
import pandas as pd
from pathlib import Path
from urllib.parse import parse_qs
from dateutil import relativedelta

# prep the environment
data_dir = Path.cwd() / "data"
uat_file_name = "ssrs_week_1_2.csv"
# uat_file_name = "ssrs_week_1.csv"
uat_file = data_dir / uat_file_name
data_file = data_dir / "uat_data.pkl"
uat_start = datetime.datetime(2019, 7, 22)


In [65]:
# load the UAT runtime data
if uat_file.exists():
    raw_df = pd.read_csv(uat_file)
else:
    print(f"Processed data missing, please run the Data Prep notebook first.")
    exit(-1)


### Normalize Columns:
Normalize (rename) columns to be in line with original column names.
If different column names are produced by different extracts, add them to the mapping here.


In [66]:
column_map = {'InstanceName': 'Server', 'ItemPath': 'ReportPath', 'UserName': 'User', 'TimeStart': 'StartTime', 
              'TimeEnd': 'StopTime', 'TimeDataRetrieval': 'DurationDataRetrieval', 
              'TimeProcessing': 'DurationProcessing', 'TimeRendering': 'DurationRendering', 'Status': 'ResultStatus'}
uat_df = raw_df.rename(columns=column_map)


### Format Columns:
1. DateTime columns = StartTime, EndTime


In [67]:
uat_df['StartTime'] = pd.to_datetime(uat_df['StartTime'], infer_datetime_format=True)
uat_df['StopTime'] = pd.to_datetime(uat_df['StopTime'], infer_datetime_format=True)


### Clean Invalid Rows:
1. Rows with NULL in the following columns:
  - User
  - ReportPath
1. Rows with StartTime before the start of UAT

  

In [68]:
# clean invalid the rows
uat_df.dropna(subset=['User', 'ReportPath'], inplace=True)

# clean pre-UAT rows
uat_df = uat_df[uat_df['StartTime'] >= uat_start]


### Create Derived Fields:
1. DurationTotal - the sum of the 3 duration columns
1. DurationTotalSec - converting DurationTotal to seconds
1. DurationDataRetrievalSec - converting DurationDataRetrieval to seconds
1. DurationProcessingSec - converting DurationProcessing to seconds
1. DurationRenderingSec - converting DurationRendering to seconds
1. ReportGroup - the broad group of users (directory)
1. ReportName - the name portion of the report
1. Agency - the Agency of the user who ran the report
1. Dims - report on how many times report execution called a /*Dim record 
   (then remove the /*Dim rows from the DataFrame)


In [69]:
# create derived fields
uat_df['DurationTotal'] = uat_df['DurationDataRetrieval'] + uat_df['DurationProcessing'] + uat_df['DurationRendering']
uat_df['DurationTotalSec'] = uat_df['DurationTotal'] / 1000
uat_df['DurationDataRetrievalSec'] = uat_df['DurationDataRetrieval'] / 1000
uat_df['DurationProcessingSec'] = uat_df['DurationProcessing'] / 1000
uat_df['DurationRenderingSec'] = uat_df['DurationRendering'] / 1000
uat_df[['ReportGroup', 'ReportName']] = uat_df['ReportPath'].str.extract('^/(?P<UserGroup>.*)/(?P<ReportName>.*$)')[['UserGroup', 'ReportName']]

# set field types
uat_df['ReportGroup'] = uat_df['ReportGroup'].astype('category')
uat_df['ReportName'] = uat_df['ReportName'].astype('category')

# uat_df.User.replace('administrator', 'Administrator', inplace=True)
# uat_df.User.replace('Administrator', 'UATUserDARe', inplace=True)
# uat_df.User.replace('REPORTAD\\UATUserOrca', 'REPORTAD\\UATUserORCA', inplace=True)
# Attempt to parse out the Agency from the username
uat_df['Agency'] = uat_df['User'].str.extract('.*UATUser(?P<Agency>.*)$')
# Add Agency of known users, explicitly
dare_users = ['REPORTAD\\UATUserOrca', 'REPORTAD\\UATUserOrca2', 'Administrator', 'administrator', 
              'REPORTAD\\ClaireC', 'REPORTAD\\RafaelE', 'REPORTAD\\RodrigoB']
uat_df.loc[uat_df['User'].isin(dare_users), 'Agency'] = 'DARe'

# Filter out Dim records
dim_list = ['/AgencyDim', '/BranchDim', '/CardGroupDim','/InstitutionDim', '/ParticipantDim']
dim_df = uat_df[uat_df.ReportPath.isin(dim_list)]
uat_df = uat_df[~uat_df.ReportPath.isin(dim_list)]
print(f"Filtered out {len(dim_df)} /*Dim records, making an average of {(len(dim_df) / len(uat_df)):.2f} /*Dim calls per ExecutionId")


Filtered out 2989 /*Dim records, making an average of 2.05 /*Dim calls per ExecutionId


### Process the Query Parameters

In [70]:
param_num_institutions = lambda x: len(parse_qs(x)['InstitutionDimKey']) if type(x) == str and 'InstitutionDimKey' in parse_qs(x).keys() else 0

def param_query_months(param_string):
    date_format = '%m/%d/%Y %I:%M:%S %p'
    if type(param_string) != str:
        return None
    params = parse_qs(param_string)
    if 'FromDate' not in params.keys() or 'ToDate' not in params.keys():
        return None
    rdelta = relativedelta.relativedelta(datetime.datetime.strptime(params['ToDate'][0], date_format),
                                         datetime.datetime.strptime(params['FromDate'][0], date_format))
    if rdelta.days > 16:
        rdelta.months += 1
    months = rdelta.years * 12 + rdelta.months
    return months

uat_df['ParamNumInstitutions'] = uat_df['Parameters'].apply(param_num_institutions)
uat_df['ParamNumMonths'] = uat_df['Parameters'].apply(param_query_months)

### Save the processed DataFrame

In [71]:
# save the loaded data
uat_df.to_pickle(data_file)
print(f"Updated {data_file} on {datetime.datetime.now()}")

Updated /Users/mcclure/PycharmProjects/dare_analytics/data/uat_data.pkl on 2019-08-08 13:58:08.111120
